imports

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from numpy import vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [3]:
import torch
torch.manual_seed(100)
np.random.seed(0)

In [4]:
data_path = '../data/'

In [7]:
!ls ../data/

augmented_module1_82.csv
data_labs_nolastvisit_aettplus1_med1.csv
test_original_data.csv
train_data_breast.csv
v2augmented_module2.csv


read data

In [10]:
train_data = pd.read_csv(data_path+ 'train_data_breast.csv')
test_data = pd.read_csv(data_path + 'test_data_breast.csv')

In [11]:
train_data.shape

(6566, 165)

In [12]:
train_data.head()

,People,Visit,bad_event,Treatment_0,Treatment_1,Treatment_2,Treatment_3,Treatment_4,Medication_0,Medication_1,...,AE_t_72,AE_t_86,AE_t_91,AE_t_92,AE_t_93,AE_t_96,AE_t_97,AE_t_98,AE_t_108,AE_t_other
0,0,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,2,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,3,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,4,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
test_data.head()

,People,Visit,bad_event,Treatment_0,Treatment_1,Treatment_2,Treatment_3,Treatment_4,Medication_0,Medication_1,...,AE_t_72,AE_t_86,AE_t_91,AE_t_92,AE_t_93,AE_t_96,AE_t_97,AE_t_98,AE_t_108,AE_t_other
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,4,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
binary_real_probability=[]
for i in range(0, 101):
  binary_real_probability.append(train_data['Medication_'+str(i)].sum()/train_data.shape[0])

read augmented med training, val data

In [15]:
m1_train = pd.read_csv(data_path+'augmented_module1_82.csv')

In [16]:
m1_train.shape

(6566, 164)

In [17]:
m1_train

,People,Visit,Treatment_0,Treatment_1,Treatment_2,Treatment_3,Treatment_4,Medication_0,Medication_1,Medication_2,...,AE_72,AE_86,AE_91,AE_92,AE_93,AE_96,AE_97,AE_98,AE_108,AE_other
0,0,0,0,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0,2,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,3,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,4,0,0,1,1,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6561,975,4,0,0,1,1,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
6562,975,5,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
6563,975,6,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
6564,975,7,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
m1_train = m1_train.set_index(['People', 'Visit'])
train_data = train_data.set_index(['People', 'Visit'])

In [19]:
m1_train = m1_train.merge(train_data['bad_event'], how= 'inner', left_index=True, right_index=True)

In [20]:
m1_train['bad_event'].unique()

array([0, 1])

In [21]:
m1_train.shape

(6566, 163)

In [22]:
m1_train.head()

Treatment_0  Treatment_1  Treatment_2  Treatment_3  Treatment_4  \
People Visit                                                                    
0      0                0            1            1            0            0   
       1                0            1            1            0            0   
       2                0            1            1            0            0   
       3                0            1            1            0            0   
       4                0            0            1            1            1   

              Medication_0  Medication_1  Medication_2  Medication_3  \
People Visit                                                           
0      0                 0             1             1             0   
       1                 0             0             1             0   
       2                 0             0             1             1   
       3                 0             0             1             1   
       4                 0             0             1             0   

              Medication_4  ...  AE_86  AE_91  AE_92  AE_93  AE_96  AE_97  \
People Visit                ...                                             
0      0                 0  ...      0      0      0      0      0      0   
       1                 0  ...      0      0      0      0      0      0   
       2                 0  ...      0      0      0      0      0      0   
       3                 0  ...      0      0      0      0      0      0   
       4                 0  ...      0      0      0      0      0      0   

              AE_98  AE_108  AE_other  bad_event  
People Visit                                      
0      0          0       0         1          0  
       1          0       0         1          0  
       2          0       0         1          0  
       3          0       0         1          0  
       4          0       0         0          0  

[5 rows x 163 columns]

In [23]:
binary_syn_probability=[]
for i in range(0, 101):
  binary_syn_probability.append(m1_train['Medication_'+str(i)].sum()/m1_train.shape[0])

In [24]:
m2_train = pd.read_csv(data_path+'v2augmented_module2.csv')

In [25]:
m2_train.shape

(6566, 164)

In [26]:
m2_train.head()

,People,Visit,Treatment_0,Treatment_1,Treatment_2,Treatment_3,Treatment_4,AE_t_0,AE_t_1,AE_t_2,...,Medication_nxt_91,Medication_nxt_92,Medication_nxt_93,Medication_nxt_94,Medication_nxt_95,Medication_nxt_96,Medication_nxt_97,Medication_nxt_98,Medication_nxt_99,Medication_nxt_100
0,0,0,0,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,3,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,4,0,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
m2_train = m2_train.set_index(['People', 'Visit'])

In [28]:
m2_train = m2_train.merge(train_data['bad_event'], how= 'inner', left_index=True, right_index=True)

In [29]:
m2_train.shape

(6566, 163)

combine

In [31]:
m1_train = m1_train.iloc[:, 0: 106]

In [32]:
m2_train = m2_train.iloc[:, 5:61]

In [33]:
m1_2_train = m1_train.merge(m2_train, how = 'inner', left_index = True, right_index = True)

In [34]:
m1_2_train.shape

(6566, 162)

personalized histogram

In [35]:
m1_2_train.reset_index(inplace=True)

/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/2109954244.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  m1_2_train.reset_index(inplace=True)
/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/2109954244.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  m1_2_train.reset_index(inplace=True)


In [36]:
train_data.reset_index(inplace=True)

/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/157586294.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data.reset_index(inplace=True)
/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/157586294.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data.reset_index(inplace=True)


combine with bad event

In [37]:
m1_2_train = m1_2_train.set_index(['People', 'Visit'])
train_data = train_data.set_index(['People', 'Visit'])

In [38]:
m1_2_train = m1_2_train.merge(train_data['bad_event'], how= 'inner', left_index=True, right_index=True)

In [39]:
m1_2_train.head()

Treatment_0  Treatment_1  Treatment_2  Treatment_3  Treatment_4  \
People Visit                                                                    
0      0                0            1            1            0            0   
       1                0            1            1            0            0   
       2                0            1            1            0            0   
       3                0            1            1            0            0   
       4                0            0            1            1            1   

              Medication_0  Medication_1  Medication_2  Medication_3  \
People Visit                                                           
0      0                 0             1             1             0   
       1                 0             0             1             0   
       2                 0             0             1             1   
       3                 0             0             1             1   
       4                 0             0             1             0   

              Medication_4  ...  AE_t_86  AE_t_91  AE_t_92  AE_t_93  AE_t_96  \
People Visit                ...                                                
0      0                 0  ...        0        0        0        0        0   
       1                 0  ...        0        0        0        0        0   
       2                 0  ...        0        0        0        0        0   
       3                 0  ...        0        0        0        0        0   
       4                 0  ...        0        0        0        0        0   

              AE_t_97  AE_t_98  AE_t_108  AE_t_other  bad_event  
People Visit                                                     
0      0            0        0         0           0          0  
       1            0        0         0           0          0  
       2            0        0         0           0          0  
       3            0        0         0           0          0  
       4            0        0         0           0          0  

[5 rows x 163 columns]

In [40]:
ae_cols = [col for col in m1_2_train if col.startswith('AE_t_')]

In [41]:
binary_ae_prob=[]
for i in ae_cols:
  binary_ae_prob.append(train_data[i].sum()/train_data.shape[0])

In [42]:
binary_syn_prob=[]
for i in ae_cols:
  binary_syn_prob.append(m1_2_train[i].sum()/m1_2_train.shape[0])

In [43]:
syn_df = m1_2_train.copy()#set_index(['People', 'Visit'])

In [44]:
train_data_bad_event = train_data['bad_event']
test_data_bad_event= test_data['bad_event']
syn_data_bad_event = syn_df['bad_event']

In [45]:
syn_df = syn_df.drop(columns='bad_event')

In [46]:
train_data= train_data.drop(columns='bad_event')

In [47]:
test_data = test_data.drop(columns='bad_event')

In [48]:
len(test_data.columns), len(train_data.columns), len(syn_df.columns)

(164, 162, 162)

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

create labels

In [51]:
m1_2_train.reset_index(inplace=True)

/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/2109954244.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  m1_2_train.reset_index(inplace=True)
/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/2109954244.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  m1_2_train.reset_index(inplace=True)


In [52]:
labels=[]

In [53]:

for i in m1_2_train.People.unique():
  temp = m1_2_train[m1_2_train['People']==i]
  if len(temp.bad_event.unique()) >1:
    labels.append(1)
  else:
    labels.append(temp.bad_event.unique()[0])


In [54]:
sum(labels)

113

In [55]:
train_data = pd.merge(train_data, train_data_bad_event, how = 'inner', left_index=True, right_index=True)

In [56]:
train_data.reset_index(inplace=True)

/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/157586294.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data.reset_index(inplace=True)
/var/folders/8v/rptqqw153dvg7g435f24xms40000gn/T/ipykernel_1413/157586294.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data.reset_index(inplace=True)


In [57]:
#comment this when training RNN with augmented data
for i in train_data.People.unique():
  temp = train_data[train_data['People']==i]
  if len(temp.bad_event.unique()) >1:
    labels.append(1)
  else:
    labels.append(temp.bad_event.unique()[0])

In [58]:
len(labels)

1742

In [59]:
test_data= pd.merge(test_data, test_data_bad_event, how = 'inner', left_index=True, right_index=True)

In [60]:
test_data.reset_index(inplace=True)

In [61]:
test_labels=[]
for i in test_data.People.unique():
  temp = test_data[test_data['People']==i]
  if len(temp.bad_event.unique()) >1:
    test_labels.append(1)
  else:
    test_labels.append(temp.bad_event.unique()[0])

In [62]:
len(test_labels)

100

In [63]:
all_train = pd.concat([train_data, m1_2_train], ignore_index=True)

In [64]:
all_train.shape

(13132, 165)

In [65]:
all_train.head()

,People,Visit,Treatment_0,Treatment_1,Treatment_2,Treatment_3,Treatment_4,Medication_0,Medication_1,Medication_2,...,AE_t_86,AE_t_91,AE_t_92,AE_t_93,AE_t_96,AE_t_97,AE_t_98,AE_t_108,AE_t_other,bad_event
0,0,0,0,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0,2,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0,3,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,4,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


ae has different vocabulary as we combined many rare AEs

In [66]:
train_data.columns

Index(['People', 'Visit', 'Treatment_0', 'Treatment_1', 'Treatment_2',
       'Treatment_3', 'Treatment_4', 'Medication_0', 'Medication_1',
       'Medication_2',
       ...
       'AE_t_86', 'AE_t_91', 'AE_t_92', 'AE_t_93', 'AE_t_96', 'AE_t_97',
       'AE_t_98', 'AE_t_108', 'AE_t_other', 'bad_event'],
      dtype='object', length=165)

In [67]:
ae_cols = [col for col in train_data if col.startswith('AE_')]
voc_ae_word2idx = {ae_cols[i]: i for i in range(len(ae_cols))}
voc_ae_word2idx['PAD'] = 0

In [68]:
voc_ae_word2idx = dict(sorted(voc_ae_word2idx.items(), key=lambda item: item[1]))

format main data and augmented data for **pytrial**

In [69]:
train_data.head()

,People,Visit,Treatment_0,Treatment_1,Treatment_2,Treatment_3,Treatment_4,Medication_0,Medication_1,Medication_2,...,AE_t_86,AE_t_91,AE_t_92,AE_t_93,AE_t_96,AE_t_97,AE_t_98,AE_t_108,AE_t_other,bad_event
0,0,0,0,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0,2,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0,3,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,4,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [70]:
visits = []
treatment_cols =  [col for col in train_data if col.startswith('Treatment_')]
medication_cols = [col for col in train_data if col.startswith('Medication_')]
ae_cols = [col for col in train_data if col.startswith('AE_')]
for i in train_data.People.unique():
  sample=[]
  temp = train_data[train_data['People']==i]
  for index, row in temp.iterrows():
    visit=[]
    visit.append(np.nonzero(row[treatment_cols].to_list())[0].tolist())
    visit.append(np.nonzero(row[medication_cols].to_list())[0].tolist())
    visit.append(np.nonzero(row[ae_cols].to_list())[0].tolist())
    sample.append(visit)
  visits.append(sample)



In [71]:
visits_aug = []
treatment_cols =  [col for col in train_data if col.startswith('Treatment_')]
medication_cols = [col for col in train_data if col.startswith('Medication_')]
ae_cols = [col for col in train_data if col.startswith('AE_')]
for i in m1_2_train.People.unique():
  sample=[]
  temp = m1_2_train[m1_2_train['People']==i]
  for index, row in temp.iterrows():
    visit=[]
    visit.append(np.nonzero(row[treatment_cols].to_list())[0].tolist())
    if len(np.nonzero(row[medication_cols].to_list())[0].tolist())!=0:
      visit.append(np.nonzero(row[medication_cols].to_list())[0].tolist())
    else:
      visit.append([0])
    visit.append(np.nonzero(row[ae_cols].to_list())[0].tolist())
    sample.append(visit)
  visits_aug.append(sample)

sanity check

In [72]:
visits[2]

[[[1, 2], [1, 12], [1, 7, 8, 9]],
 [[1, 2], [1, 12], [1, 10]],
 [[1, 2], [1, 12], [1, 10]],
 [[1, 2], [1], [1, 10, 26]],
 [[2, 3, 4], [12], [1, 9, 26]],
 [[2, 3, 4], [12], [1, 4]],
 [[2, 3, 4], [12], [1, 3, 4, 7]],
 [[0], [100], [0]],
 [[0], [100], [0]],
 [[0], [100], [0]]]

In [73]:
visits_aug[2]

[[[1, 2], [1], [1, 3, 7]],
 [[1, 2], [1, 41], [1, 10]],
 [[1, 2], [1, 41], [1, 10]],
 [[1, 2], [1], [1, 10]],
 [[2, 3, 4], [0], [1]],
 [[2, 3, 4], [0], [1, 4]],
 [[2, 3, 4], [0], [1, 3, 4, 7]],
 [[0], [100], [0]],
 [[0], [100], [0]],
 [[0], [100], [0]]]

add augmented data to main data

In [74]:
#main data + augmented data
visits.extend(visits_aug)

In [75]:
len(visits)

1742

format test data for pytrial

In [76]:
visits_test = []
treatment_cols =  [col for col in train_data if col.startswith('Treatment_')]
medication_cols = [col for col in train_data if col.startswith('Medication_')]
ae_cols = [col for col in train_data if col.startswith('AE_')]
for i in test_data.People.unique():
  sample=[]
  temp = test_data[test_data['People']==i]
  for index, row in temp.iterrows():
    visit=[]
    visit.append(np.nonzero(row[treatment_cols].to_list())[0].tolist())
    if len(np.nonzero(row[medication_cols].to_list())[0].tolist())!=0:
      visit.append(np.nonzero(row[medication_cols].to_list())[0].tolist())
    else:
      visit.append([0])
    visit.append(np.nonzero(row[ae_cols].to_list())[0].tolist())
    sample.append(visit)
  visits_test.append(sample)

**find max visit**

In [77]:
max_visit=-1
for i in visits:
  if len(i)> max_visit:
    max_visit = len(i)
  
for i in visits_test:
  if len(i)>max_visit:
    max_visit = len(i)

In [78]:
max_visit

13

**pytrial rnn**

In [102]:
#!pip install git+https://github.com/RyanWangZf/pytrial.git

In [79]:
from pytrial.tasks.indiv_outcome.data import TabularPatient, SequencePatient

In [80]:
from pytrial.data.demo_data import load_trial_patient_sequence

In [81]:
z= load_trial_patient_sequence()


 Download trial patient sequence data to ./demo_data/demo_patient_sequence/trial.
#####Demo Data Folder#####
['timestamp.pkl', 'binary_feature.txt', 'seq_patient_nct00174655.zip', 'visit_stage.pkl', 'visit.pkl', 'voc.pkl', 'process_NCT00174655.ipynb', 'numerical_feature.txt', 'feature.csv']
####################


/Users/trishadas/opt/anaconda3/lib/python3.9/site-packages/pytrial/data/patient_data.py:92: UserWarning: No metadata provided. Metadata will be automatically detected from your data. This process may not be accurate. We recommend writing metadata to ensure correct data handling.
  warnings.warn('No metadata provided. Metadata will be automatically '


In [82]:
# create vocabulary
class Voc(object):
    def __init__(self, word2idx):
        self.word2idx = word2idx # default padding index is 0
        self.idx2word = {self.word2idx[x]:x for x in self.word2idx}
    
    def __len__(self):
        return len(self.idx2word.keys())

    def add_sentence(self, sentence):
        for word in sentence:
            if word not in self.word2idx:
                self.idx2word[len(self.word2idx)] = word
                self.word2idx[word] = len(self.word2idx)

In [83]:
voc_ae = Voc(voc_ae_word2idx)

In [84]:
voc ={ 'treatment': z['voc']['treatment'], 'medication' : z['voc']['medication'], 'adverse_event': voc_ae }

In [85]:

seqdata = SequencePatient(
    data={'v':visits, 'y': labels, 'x':None},
    metadata={
        'visit':{'mode':'dense', 'order': ['treatment','medication','adverse_event']},
        'label':{'mode':'tensor'},
        'voc':voc,
        'max_visit':13,
        }
    )

# build the test data
val_seqdata = SequencePatient(
    data={'v': visits_test, 'y':test_labels, 'x':None},
    metadata={
        'visit':{'mode':'dense', 'order': ['treatment','medication','adverse_event']},
        'label':{'mode':'tensor'},
        'voc':voc,
        'max_visit':13,
        }
    )

In [86]:
from pytrial.data.patient_data import SeqPatientCollator # we need a collation function to process the input SequencePatient dataset
from pytrial.tasks.indiv_outcome.sequence import RNN

: 

: 

In [111]:
[len(voc[o]) for o in ['treatment','medication','adverse_event']]

[5, 101, 56]

In [112]:

model = RNN(
    vocab_size=[len(voc[o]) for o in ['treatment','medication','adverse_event']], # get the vocab size for each type of events to build the event embedding layer
    orders=['treatment','medication','adverse_event'], # similar, we need an order
    mode='binary',
    max_visit=13,
    n_rnn_layer = 3,
    bidirectional=False,
    epochs=200,
    batch_size=200,
    device='cpu',
)

#random.seed(42)
model.fit(train_data=seqdata, valid_data=val_seqdata)

{'lr': 0.0001, 'weight_decay': 0.0001}
***** Running training *****
  Num examples = 871
  Num Epochs = 200
  Total optimization steps = 1000


Training Epoch:  10%|█         | 20/200 [00:13<01:36,  1.86it/s]


######### Train Loss 100 #########
0 0.5884 


######### Eval 100 #########
auc: 0.5690
New best score: from -inf to 0.568986568986569
Best checkpoint is updated at 100 with auc 0.568986568986569.



Training Epoch:  20%|██        | 40/200 [00:24<01:54,  1.40it/s]


######### Train Loss 200 #########
0 0.3882 


######### Eval 200 #########
auc: 0.6007
New best score: from 0.568986568986569 to 0.6007326007326007
Best checkpoint is updated at 200 with auc 0.6007326007326007.


...


######### Train Loss 800 #########
0 0.3374 


######### Eval 800 #########
auc: 0.7375



Training Epoch:  90%|█████████ | 180/200 [01:51<00:10,  1.96it/s]


######### Train Loss 900 #########
0 0.3364 


######### Eval 900 #########
auc: 0.7350



Training Epoch: 100%|██████████| 200/200 [02:05<00:00,  1.59it/s]


######### Train Loss 1000 #########
0 0.3310 


######### Eval 1000 #########
auc: 0.7302
Load best ckpt from `./checkpoints/best`.
Training completes.


In [113]:
model.load_model('./checkpoints/best')

In [114]:
res = model.predict(val_seqdata)


In [115]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_labels, res.squeeze())

0.78998778998779

In [116]:
#model.save_model('./checkpoints/indiv_outcome.sequence/')